In [31]:
import numpy as np
import math
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score


In [32]:
Train_Dataset = pd.read_csv("/home/barath/Documents/DS/Python3/Logistic/Loanpredtiction/train_ctrUa4K.csv")
Test_Dataset = pd.read_csv("/home/barath/Documents/DS/Python3/Logistic/Loanpredtiction/test_lAUu6dG.csv")

del Train_Dataset['Loan_ID']
del Train_Dataset['Loan_Status']
del Test_Dataset['Loan_ID']

In [33]:
Dataset = pd.concat([Train_Dataset, Test_Dataset], axis=0,ignore_index=True)

In [34]:
NullFree_Dataset = pd.DataFrame()
def fill_na(dataset):
    for col in dataset.columns:
        if dataset[col].isnull().sum() == 0:
            NullFree_Dataset[dataset[col].name]= dataset[col]
        elif dataset[col].value_counts().count() <= 4:
            NullFree_Dataset[dataset[col].name] = dataset[col].fillna(dataset[col].value_counts().index[0])
        else:
            NullFree_Dataset[dataset[col].name] = dataset[col].fillna(np.floor(dataset[col].value_counts().mean()))
    return NullFree_Dataset
Dataset = fill_na(Dataset)
#Dataset

In [35]:
cat_vars = pd.DataFrame()
num_vars = pd.DataFrame()
fac_data = pd.DataFrame()
le = LabelEncoder()
def splitcatvars(dataset):
    for col in dataset.columns:
        if dataset[col].value_counts().count()  <=4:
            cat_vars[dataset[col].name] = dataset[col]
        else:
            num_vars[dataset[col].name] = dataset[col]
   
    return cat_vars, num_vars
cat_vars, num_vars = splitcatvars(Dataset)

def tonumeric(dataset):
    for col in dataset.columns:
        fac_data[dataset[col].name] = le.fit_transform(dataset[col])
    return fac_data
New_Dataset = tonumeric(cat_vars)
print("New_Dataset:",New_Dataset.shape,' ',"Num_vars:", num_vars.shape)

New_Dataset: (981, 7)   Num_vars: (981, 4)


In [36]:
Dataframe = pd.concat([New_Dataset, num_vars],axis=1)

In [37]:
Trainset = Dataframe.iloc[:614,:]
Testset = Dataframe.iloc[614:,:]
print("Trainset:",Trainset.shape,' ',"Testset:", Testset.shape)

Trainset: (614, 11)   Testset: (367, 11)


In [38]:
Train_Set = pd.read_csv("/home/barath/Documents/DS/Python3/Logistic/Loanpredtiction/train_ctrUa4K.csv")
Target = Train_Set['Loan_Status'].map(lambda i: 1 if i=='Y' else 0)
#Target

In [39]:
lorm = LogisticRegression()

In [40]:
sacled_data = StandardScaler().fit_transform(Trainset)
log_clf = lorm.fit(sacled_data, Target)

In [41]:
Train_pred = log_clf.predict(sacled_data)

In [42]:
con_mat = confusion_matrix(Target, Train_pred)
acc_scr = accuracy_score(Target, Train_pred)
print("Confusion_Matrix:",con_mat,'\n',"Accuracy_Score:",acc_scr)

Confusion_Matrix: [[ 84 108]
 [  8 414]] 
 Accuracy_Score: 0.8110749185667753


In [43]:
scaled_data1 = StandardScaler().fit_transform(Testset)
Test_pred = log_clf.predict(scaled_data1)
acc_scr1 = cross_val_score(log_clf,scaled_data1, Test_pred)
print("Accuracy_Score:",acc_scr1.mean())

Accuracy_Score: 0.9945205479452055


In [44]:
Test_ds = pd.read_csv("/home/barath/Documents/DS/Python3/Logistic/Loanpredtiction/test_lAUu6dG.csv")
outcome = pd.DataFrame()
outcome['Loan_ID'] = Test_ds['Loan_ID']

for i in Test_pred:
    outcome['Loan_Status']=np.vectorize(lambda s: 'Y' if s==1 else 'N')(Test_pred) 
outcome[['Loan_ID','Loan_Status']].to_csv('/home/barath/Documents/DS/Python3/Logistic/Loanpredtiction/outcome.csv',index=False)